# Resources

1. need to see this [mbert](https://huggingface.co/sagorsarker/mbert-bengali-ner) for getting idea about architecture [mbert full code](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/token_classification.ipynb)

### alternate implementation

https://saturncloud.io/blog/implementing-huggingface-bert-using-tensorflow-for-sentence-classification/

### prev codes

1. https://colab.research.google.com/drive/1RjVnXwyBlwGjMozF8is_Yqxw06pbCdNi?usp=sharing

2. https://colab.research.google.com/drive/1v9gQS2evy-KoFN5PPxlRim7bmXiJhLK5?usp=sharing

3. https://colab.research.google.com/drive/16qmrSRdDNnW53trCBk1c19L3rrAzuMtB?usp=sharing

4. https://colab.research.google.com/drive/1pS9azT0kPm5uAPhFXZ9WPRNcngYOk0MW?usp=sharing

5. https://colab.research.google.com/drive/1VOQkncOEtPkj91TbGl9DIFScXOs8lguG?usp=sharing

# Token classification (TensorFlow)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!apt install git-lfs
# for kaggle
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git

You will need to setup git, adapt your email and name in the following cell.

In [1]:
!git config --global user.email "imtiazahmed5573@gmail.com"
!git config --global user.name "imtiaz114"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

* huggingace prob in [kaggle](https://stackoverflow.com/questions/76363436/cannot-import-name-partialstate-from-accelerate-when-using-huggingface-pipel) solve

In [2]:
from huggingface_hub import notebook_login

notebook_login()

# Dataset

In [56]:
import pandas as pd
dataset_path = "/kaggle/input/hishab-dataset-person-extractor/hishab_ds/hishab_ds/concatenated_bangla_ner_multiclass.csv"
df = pd.read_csv(dataset_path,index_col=False)
df.head(5)

,tokens,tags
0,"['\ufeffউন্নয়নের', 'বিস্ময়', 'বাংলাদেশ', 'বাংল...","['O', 'O', 'B-LOC', 'B-LOC', 'O', 'O', 'O', 'O..."
1,"['অর্থনীতি', 'ও', 'আর্থসামাজিক', 'বেশির', 'ভাগ...","['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', '..."
2,"['নিম্ন', 'আয়ের', 'দেশগুলোকে', 'ছাড়িয়েছে', 'তো...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']"
3,"['আন্তর্জাতিক', 'মুদ্রা', 'তহবিল', '(', 'আইএমএ...","['B-ORG', 'I-ORG', 'I-ORG', 'O', 'B-ORG', 'O',..."
4,"['সবাইকে', 'অন্তর্ভুক্ত', 'করে', 'প্রবৃদ্ধি', ...","['O', 'O', 'O', 'O', 'O', 'O', 'B-LOC', 'O', '..."


In [57]:
# take list of list dataset
tokens = df["tokens"].tolist()
tags   = df["tags"].tolist()
print(len(tokens),len(tags)) # taking all the lines here

9926 9926


In [58]:
# tags and tokens fixing
newTags = []
for idx in range(len(tags)):
    demo = tags[idx]
    # getting individual tokens
    demo = demo.replace("[", "" )# at first tabs necessary to add count
    demo = demo.replace("]", "" )# at first tabs necessary to add count
    demo = demo.replace("'", "" )# at first tabs necessary to add count
    demo = demo.replace(" ", "" )# at first tabs necessary to add count
    demo = demo.split(",")
    
    for j in range(len(demo)):
        if demo[j] == "B-PER" or  demo[j] == "I-PER":
            demo[j] = "B-PER"
        else:
            demo[j] = "O"

    newTags.append(demo)
    
tags = newTags
#############
newTokens = []
for idx in range(len(tokens)):
    demo = tokens[idx]
    # getting individual tokens
    demo = demo.replace("[", "" )# at first tabs necessary to add count
    demo = demo.replace("]", "" )# at first tabs necessary to add count
    demo = demo.replace("'", "" )# at first tabs necessary to add count
    demo = demo.replace(" ", "" )# at first tabs necessary to add count
    demo = demo.split(",")

    newTokens.append(demo)
tokens = newTokens
print(len(tokens),len(tags))

9926 9926


In [59]:
print(tokens[0][0])
print(tags[0][0])
# so tags and tokens are fixed

\ufeffউন্নয়নের
O


In [60]:
print(tokens[0])

['\\ufeffউন্নয়নের', 'বিস্ময়', 'বাংলাদেশ', 'বাংলাদেশের', 'অগ্রগতি', 'উদাহরণ', 'দেওয়ার', 'মতোই', '।']


In [61]:
print(tags[0])

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [62]:
import itertools
# Flatten the list of lists into a single list
flattened_list = list(itertools.chain(*tags))
# Get the unique elements using set
label_names = list(set(flattened_list))
label_name = label_names
label_names# checking unique tag names

['O', 'B-PER']

In [63]:
for i in range(len(tags)):
    demo =tags[i]
    for j in range(len(demo)):
        demo[j] = label_name.index(demo[j])
    tags[i] = demo

print(tags[:5]) # converting tags to int

[[0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [64]:
words = tokens[0]
labels = tags[0]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_name[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)
# simulataneous checking of tokens and labels

\ufeffউন্নয়নের বিস্ময় বাংলাদেশ বাংলাদেশের অগ্রগতি উদাহরণ দেওয়ার মতোই । 
O              O      O        O          O       O      O      O    O 


In [65]:
from transformers import AutoTokenizer

model_checkpoint = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# initializing classifier

In [66]:
tokenizer.is_fast

True

In [67]:
inputs = tokenizer(tokens[0], is_split_into_words=True)
inputs.tokens()
# using tokenizer to pad suitably # here some words are broke down

['<s>',
 '▁',
 '\\',
 'uf',
 'eff',
 'উ',
 'ন্ন',
 'য়',
 'নের',
 '▁বি',
 'স্',
 'ময়',
 '▁বাংলাদেশ',
 '▁বাংলাদেশের',
 '▁অগ্র',
 'গ',
 'তি',
 '▁উ',
 'দা',
 'হ',
 'রণ',
 '▁দেওয়া',
 'র',
 '▁মতো',
 'ই',
 '▁।',
 '</s>']

In [68]:
print(inputs.word_ids()) # this tokenization is useful for bert
# need to understand the in depth theory

[None, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 3, 4, 4, 4, 5, 5, 5, 5, 6, 6, 7, 7, 8, None]


In [69]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

# labelling words and ids with tokens after adding padding tokens

In [70]:
labels = tags[0]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))
# matching with the bert input

[0, 0, 0, 0, 0, 0, 0, 0, 0]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


### creating dataset

In [71]:
# deleting the mismatched data
# reading as tab-separated file format as it would be easier for tf.data.Dataset object
def cleaning_data(tokens, tags):
    mismatch = []
    newtokens = []
    newtags   = []
    flags      = []
    newcount = 0
    for i in range(len(tokens)):
        c = 0
        if len(tokens[i]) > 0 and len(tags[i])>0:
            if len(tokens[i]) != len(tags[i]):
                mismatch.append(i)
            else:
                
                
                
                newtokens.append(tokens[i])
                newtags.append(tags[i])
                
                demo = tags[i]
                for j in demo:
                    if j == 1 or j == 2:
                        c = 1
                if c == 1:
                    flags.append(1)
                else:
                    flags.append(0)
                

                
    print("total number of flag sentences %d"%(sum(flags)))
    print(len(flags),"\n")
    print("total number of mismatched token and tag pair sentneces %d"%(len(mismatch)))
    print("total number of sentneces %d"%(i+1))
    print("final length of dataset %d"%len(newtokens))
    return newtokens,newtags,flags

tokens,tags,flags = cleaning_data(tokens, tags)

total number of flag sentences 2104
7963 

total number of mismatched token and tag pair sentneces 1963
total number of sentneces 9926
final length of dataset 7963


In [72]:
# train val test split
from sklearn.model_selection import train_test_split
# Split the data into train, validation, and test datasets
tokens_train, tokens_val, tags_train, tags_val = train_test_split(tokens, tags,stratify=flags, test_size=0.2, random_state=42)
#tokens_val, tokens_test, tags_val, tags_test = train_test_split(tokens_val, tags_val, test_size=0.25, random_state=42)

# Print the split datasets
print("Train dataset:")
print(tokens_train[0])
print(tags_train[0])
print(len(tokens_train),"\n")

print("Validation dataset:")
print(tokens_val[0])
print(tags_val[0])
print(len(tokens_val),"\n")

#print("Test dataset:")
#print(tokens_test[0])
#print(tags_test[0])
#print(len(tokens_test),"\n")

Train dataset:
['বিবিসির', 'প্রতিবেদনে', 'বলা', 'হয়েছে', '<PUNCT>', '২০১৪', 'সালের', 'শুমারি', 'অনুযায়ী', 'ভারতে', 'বাঘের', 'সংখ্যা', 'এখন', '২', 'হাজার', '২২৬টি', '<PUNCT>']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
6370 

Validation dataset:
['স্বাধীনতার', 'সপক্ষের', 'দল', 'আওয়ামী', 'লীগের', 'সঙ্গে', 'থেকে', 'তিনি', 'রাজনীতি', 'করতে', 'চান', '।']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
1593 



In [73]:
import gc
gc.collect()

18

In [74]:
# converting tokens and tags to datasetdict
from datasets import Dataset, DatasetDict

# Create Dataset objects for train, validation, and test splits
train_dataset = Dataset.from_dict({'tokens': tokens_train, 'tags': tags_train})
validation_dataset = Dataset.from_dict({'tokens': tokens_val, 'tags': tags_val})
#test_dataset = Dataset.from_dict({'tokens': tokens_test, 'tags': tags_test})

# Create a DatasetDict object
raw_datasets = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    #'test': test_dataset
})

In [75]:
raw_datasets["train"].column_names

['tokens', 'tags']

In [76]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [77]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [78]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(
    tokenizer=tokenizer, return_tensors="tf"
)

In [79]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


<tf.Tensor: shape=(2, 33), dtype=int64, numpy=
array([[-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0, -100],
       [-100,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0, -100, -100, -100, -100]])>

In [80]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]


In [81]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=8,#16
)

tf_eval_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels", "token_type_ids"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=8,#16
)

### dictionaries

In [82]:
label_names

['O', 'B-PER']

In [83]:
prevnames = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC']

In [84]:
id2label = {i: label for i, label in enumerate(prevnames)}
label2id = {v: k for k, v in id2label.items()}

In [85]:
id2label

{0: 'O',
 1: 'B-PER',
 2: 'I-PER',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC'}

In [86]:
gc.collect()

736

In [87]:
from transformers import TFAutoModelForTokenClassification

# Initialize the model
model = TFAutoModelForTokenClassification.from_pretrained(
    "xlm-roberta-base",
    id2label=id2label,
    label2id=label2id,
)

All PyTorch model weights were used when initializing TFXLMRobertaForTokenClassification.

Some weights or buffers of the TF 2.0 model TFXLMRobertaForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [88]:
model.config.num_labels

7

In [89]:
#from huggingface_hub import notebook_login

#notebook_login()

In [90]:
from transformers import create_optimizer
import tensorflow as tf

# Train in mixed-precision float16
# Comment this line out if you're using a GPU that will not benefit from this
tf.keras.mixed_precision.set_global_policy("mixed_float16")

# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_epochs = 2#5#10#
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

In [91]:
tokenizer

XLMRobertaTokenizerFast(name_or_path='xlm-roberta-base', vocab_size=250002, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False)}, clean_up_tokenization_spaces=True)

In [92]:
gc.collect()

0

In [93]:
from transformers.keras_callbacks import PushToHubCallback

dirname = "/kaggle/working/mbert-ner-baseline-1"

callback = PushToHubCallback(output_dir=dirname, tokenizer=tokenizer)

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=[callback],
    epochs=num_epochs,
)

/kaggle/working/mbert-ner-baseline-1 is already a clone of https://huggingface.co/imtiaz114/mbert-ner-baseline-1. Make sure you pull the latest changes with `repo.git_pull()`.


Epoch 1/2


/opt/conda/lib/python3.10/site-packages/tensorflow/python/framework/indexed_slices.py:459: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 192001536 elements. This may consume a large amount of memory.
  warnings.warn(


796/796 [==============================] - 201s 218ms/step - loss: 0.1386 - val_loss: 0.0633
Epoch 2/2
796/796 [==============================] - 218s 274ms/step - loss: 0.0604 - val_loss: 0.0579


In [94]:
!pip install seqeval

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [95]:
import evaluate

metric = evaluate.load("seqeval")

### evaluation

In [96]:
import numpy as np

all_predictions = []
all_labels = []
for batch in tf_eval_dataset:
    logits = model.predict_on_batch(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    for prediction, label in zip(predictions, labels):
        for predicted_idx, label_idx in zip(prediction, label):
            if label_idx == -100:
                continue
            #if label_idx > 1:
                #label_idx = 0
            #if predicted_idx > 1:
                #predicted_idx = 0
            all_predictions.append(prevnames[predicted_idx])#label_names
            all_labels.append(prevnames[label_idx])
metric.compute(predictions=[all_predictions], references=[all_labels])

{'PER': {'precision': 0.6908171861836563,
  'recall': 0.7606679035250464,
  'f1': 0.7240618101545254,
  'number': 1078},
 'overall_precision': 0.6908171861836563,
 'overall_recall': 0.7606679035250464,
 'overall_f1': 0.7240618101545254,
 'overall_accuracy': 0.9737357424043606}

# Prediction

In [97]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = dirname#"/kaggle/working/bert-finetuned-ner-baseline-1"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("প্রতিবেদক ছিলেন সাপ্তাহিকের রাজশাহীর বিভাগীয় প্রতিনিধি সায়েম সাবু ।")

Some layers from the model checkpoint at /kaggle/working/mbert-ner-baseline-1 were not used when initializing TFXLMRobertaForTokenClassification: ['dropout_113']
- This IS expected if you are initializing TFXLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFXLMRobertaForTokenClassification were initialized from the model checkpoint at /kaggle/working/mbert-ner-baseline-1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForTokenClassification for predictions without further training.


[{'entity_group': 'PER',
  'score': 0.955,
  'word': 'সায়েম',
  'start': 55,
  'end': 60},
 {'entity_group': 'PER',
  'score': 0.955,
  'word': 'সাবু',
  'start': 61,
  'end': 65}]

In [98]:
output = token_classifier("প্রতিবেদক ছিলেন সাপ্তাহিকের রাজশাহীর বিভাগীয় প্রতিনিধি সায়েম সাবু ।")
print(type(output))
output

<class 'list'>


[{'entity_group': 'PER',
  'score': 0.955,
  'word': 'সায়েম',
  'start': 55,
  'end': 60},
 {'entity_group': 'PER',
  'score': 0.955,
  'word': 'সাবু',
  'start': 61,
  'end': 65}]

In [110]:
person_names = []
current_person = ""

for entity in output:
    if entity['entity_group'] == 'PER':
        current_person += entity['word']
        person_names.append(current_person)
        current_person = ""

    else:
        if current_person:
            #person_names.append(current_person)
            current_person = ""


# Add the last person name entity if present
#if current_person:
    #person_names.append(current_person)

print(person_names)

['সায়েম', 'সাবু']


In [100]:
clean_names=[]
for demo in person_names:
    demo = demo.replace("#", "" )
    clean_names.append(demo)
clean_names

['সায়েমসাবু']

In [111]:
def gettingPersonName(inputs):
    print(inputs)
    output = token_classifier(inputs)
    
    
    person_names = []
    current_person = ""

    for entity in output:
        if entity['entity_group'] == 'PER':
            current_person += entity['word']
            person_names.append(current_person)
            current_person = ""

        else:
            if current_person:
                #person_names.append(current_person)
                current_person = ""
        
    clean_names=[]
    for demo in person_names:
        demo = demo.replace("#", "" )
        clean_names.append(demo)
    return clean_names

text = "প্রতিবেদক ছিলেন সাপ্তাহিকের রাজশাহীর বিভাগীয় প্রতিনিধি সায়েম সাবু ।"
print(gettingPersonName(text))

প্রতিবেদক ছিলেন সাপ্তাহিকের রাজশাহীর বিভাগীয় প্রতিনিধি সায়েম সাবু ।
['সায়েম', 'সাবু']


In [112]:
text = "নিরাপত্তা কর্মকর্তা জান মোহাম্মদ আদালতকে মোশাররফের ‘অসুস্থতা’ সম্পর্কে অবহিত করেন ।"
print(gettingPersonName(text))

নিরাপত্তা কর্মকর্তা জান মোহাম্মদ আদালতকে মোশাররফের ‘অসুস্থতা’ সম্পর্কে অবহিত করেন ।
['জান', 'মোহাম্মদ', 'মোশাররফের']


In [113]:
text = "এতে রয়েছে কত্থক ‘গীতমালিকা’ শীর্ষক নৃত্য পরিবেশনা।"
print(gettingPersonName(text))

এতে রয়েছে কত্থক ‘গীতমালিকা’ শীর্ষক নৃত্য পরিবেশনা।
[]


In [114]:
text = "ক্রমবর্ধমান রাজনৈতিক উত্তেজনার সঙ্গে মানবতাবিরোধী অপরাধে বিচারের বিষয়টি যুক্ত হয়ে সারা দেশে নিরাপত্তা পরিস্থিতির অবনতি হচ্ছে ।"
print(gettingPersonName(text))

ক্রমবর্ধমান রাজনৈতিক উত্তেজনার সঙ্গে মানবতাবিরোধী অপরাধে বিচারের বিষয়টি যুক্ত হয়ে সারা দেশে নিরাপত্তা পরিস্থিতির অবনতি হচ্ছে ।
[]


In [115]:
text = "প্রেমের ময়দানে ক্যারোলিন ওজনিয়াকি আর ররি ম্যাকইলরয়ের লক্ষণীয় উন্নতিই হয়েছে ।"
print(gettingPersonName(text))

প্রেমের ময়দানে ক্যারোলিন ওজনিয়াকি আর ররি ম্যাকইলরয়ের লক্ষণীয় উন্নতিই হয়েছে ।
['ক্যারোলিন', 'ওজনিয়াকি', 'ররি', 'ম্যাকইলরয়ের']


# Downloading folder

In [106]:
import zipfile
import os
from IPython.display import FileLink

def zip_dir(directory = os.curdir, file_name = 'directory.zip'):
    """
    zip all the files in a directory
    
    Parameters
    _____
    directory: str
        directory needs to be zipped, defualt is current working directory
        
    file_name: str
        the name of the zipped file (including .zip), default is 'directory.zip'
        
    Returns
    _____
    Creates a hyperlink, which can be used to download the zip file)
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))

    return FileLink(file_name)

In [107]:
zip_dir()

/kaggle/working/directory.zip

# Insight

* Bert is performing too good actually, huge overfitting is occuring, need to tackle it